In [49]:
import os
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
from torch import optim
from timeit import default_timer
import numpy as np
import pandas as pd

In [50]:
data_path = '../chest_xray'         ## chemin vers le dossier contenant les données
train_path = data_path + '/train/'  ## chemin des données d'entrainement
val_path = data_path + '/val/'      ## chemin des données de validation
test_path = data_path + '/test/'    ## chemin des données de tes

save_path = 'model.pth'

In [51]:
image_transforms = {
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),     ## changement des différentes transformations pour l'augmentation de données
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'val':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
  
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [52]:
data = {
    'train':
    datasets.ImageFolder(root=train_path, transform=image_transforms['train']),
    'val':
    datasets.ImageFolder(root=val_path, transform=image_transforms['val']),
    'test':
    datasets.ImageFolder(root=test_path, transform=image_transforms['test'])
}


In [53]:
batch_size = 128    ## choix de la taille des batchs

dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'val': DataLoader(data['val'], batch_size=batch_size, shuffle=True),
    'test': DataLoader(data['test'], batch_size=batch_size, shuffle=True)
}

In [54]:
trainiter = iter(dataloaders['train'])
features, labels = next(trainiter)

In [55]:
model = models.resnet50(pretrained=True)  ## choix du modèle pré-entrainé

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [56]:
for param in model.parameters():
    param.requires_grad = False

In [57]:
n_inputs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(n_inputs, 2), nn.LogSoftmax(dim=1))  ## choix de remplacement de la dernière couche permettant la classification de 2 classes
 
model = model.to('cuda')


ModuleAttributeError: ignored

In [ ]:
criterion = nn.NLLLoss()                                  ## choix de la fonction de perte
optimizer = optim.Adam(model.parameters(), lr = 0.05)     ## choix de l'algorithme d'optimisation de la descente de gradient

In [ ]:
nb_epochs = 10          ## choix du nombre d'epochs
history = []

for epoch in range(nb_epochs) :
    start = default_timer()

    
    train_loss = 0.0
    valid_loss = 0.0
    train_acc = 0
    valid_acc = 0

    valid_loss_min = 1000000

    model.train()
    for ii, (data, target) in enumerate(dataloaders['train']):
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        pred = torch.max(output, dim=1)[1]
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.cpu().numpy())
        accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
        train_acc += accuracy.item() * data.size(0)
        print(f'Epoch: {epoch}\t{100 * (ii + 1) / len(dataloaders["train"]):.2f}% complete. {default_timer() - start:.2f} seconds elapsed in epoch.')
    with torch.no_grad() :
        model.eval()
        for data, target in dataloaders['val'] :
            data, target = data.cuda(), target.cuda()
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item() * data.size(0)
            pred = torch.max(output, dim=1)[1]
            correct_tensor = pred.eq(target.data.view_as(pred))
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            valid_acc += accuracy.item() * data.size(0)
        train_loss = train_loss / len(dataloaders['train'].dataset)
        valid_loss = valid_loss / len(dataloaders['val'].dataset)
        train_acc = train_acc / len(dataloaders['train'].dataset)
        valid_acc = valid_acc / len(dataloaders['val'].dataset)
        history.append([train_loss, valid_loss, train_acc, valid_acc])
        if valid_loss < valid_loss_min:
                    torch.save(model.state_dict(), save_path)
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_acc
                    best_epoch = epoch
model.optimizer = optimizer
history = pd.DataFrame(history, columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])


print(history)

In [ ]:
model.load_state_dict(torch.load(save_path))
model.eval()

In [ ]:
test_loader=dataloaders['test']
test_loss = 0.0
class_correct = list(0. for i in range(2))
class_total = list(0. for i in range(2))
classes = [0,1]
model.eval()
i = 1
for data, target in test_loader:
    i=i+1
    if len(target)!=batch_size:
        continue
    data, target = data.cuda(), target.cuda()
    output = model(data)
    loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)
    _, pred = torch.max(output, 1)    
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    
    for i in range(batch_size):       
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

test_loss = test_loss/len(test_loader.dataset)
print('Loss: {:.6f}\n'.format(test_loss))

for i in range(2):
    if class_total[i] > 0:
        print('Précision de test de %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))

print('\nPrécision de test (Total): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))